In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import gc

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/dist-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/dist-packages/pandas/core/indexes/base.py:6: RuntimeWarning: numpy.dtype size changed, may indicate bi

In [2]:
%time data = pd.read_csv('data/application_train.csv')
data_submission = pd.read_csv('data/application_test.csv')
submission_id = data_submission['SK_ID_CURR']

def process_data(data):
    Null_dict = dict()

    for column in ['NAME_HOUSING_TYPE','OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE']:
        Null_dict[column] = 0


    for column in ['NAME_HOUSING_TYPE','OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE']:
        if data[column].isnull().sum()>0:
            Null_dict[column] = data[column].isnull().sum()

        if (data[column]=='XNA').sum()>0:
            Null_dict[column] = (data[column]=='XNA').sum()

    for (key,value) in Null_dict.items():
        Null_dict[key] = float(value)/data.shape[0]

    def replace_binary_categorical_var(df, column_name):
        categories = sorted(list(df[column_name].unique()))
        if np.nan in categories:
            categories.remove(np.nan)
        assert(len(categories) == 2)
        df.loc[df[column_name] == categories[0], column_name] = 0
        df.loc[df[column_name] == categories[1], column_name] = 1    
        df[column_name] = df[column_name].astype('float')
            
    for col, dtype in data.dtypes.iteritems():
        if np.issubdtype(dtype, np.number) and col.startswith('AMT_'):
            data[col + '/(AMT_CREDIT)'] = data[col] / data['AMT_CREDIT']
            
    data['FONDKAPREMONT_MODE'] = data['FONDKAPREMONT_MODE'].replace('XNA', np.nan)
    
    data['INCOME_PER_PERSON'] = data['AMT_INCOME_TOTAL'] / data['CNT_CHILDREN']
    
    # Name contract type is either Cash loans or Revolving loans
    replace_binary_categorical_var(data, 'NAME_CONTRACT_TYPE')

    # Gender is either male, female or N/A. We'll consider it binary
    data = data.replace('XNA', np.nan)
    replace_binary_categorical_var(data, 'CODE_GENDER')

    # FLAG_OWN_CAR and FLAG_OWN_REALTY are flags, either Y or N
    replace_binary_categorical_var(data, 'FLAG_OWN_CAR')
    replace_binary_categorical_var(data, 'FLAG_OWN_REALTY')

    # We'll consider unknown to be N/A
    data['NAME_FAMILY_STATUS'] = data['NAME_FAMILY_STATUS'].replace('Unknown', np.nan)

    # All these are categorical
    data = pd.get_dummies(data)

    return data

%time data = process_data(data)
data_submission = process_data(data_submission)


CPU times: user 5.54 s, sys: 370 ms, total: 5.91 s
Wall time: 5.93 s
CPU times: user 7.47 s, sys: 1.76 s, total: 9.23 s
Wall time: 9.24 s


In [3]:
%time previous_data = pd.read_csv('data/previous_application.csv')
def process_previous_application_data(previous_data):
    previous_data = pd.get_dummies(previous_data)
    previous_data.replace(365243, np.nan, inplace=True)
    
    previous_data.sort_values(['SK_ID_CURR', 'DAYS_DECISION'], ascending=[True, False], inplace=True)
    previous_data['LAST_APP_REJECTED'] = previous_data['NAME_CONTRACT_STATUS_Refused']
    previous_data.loc[previous_data['SK_ID_CURR'].diff() == 0, 'LAST_APP_REJECTED'] = np.nan
    #print(previous_data[['SK_ID_CURR', 'DAYS_DECISION', 'LAST_APP_REJECTED', 'NAME_CONTRACT_STATUS_Refused']].head(25))
    previous_data['percentage_down'] = previous_data['AMT_DOWN_PAYMENT'] / previous_data['AMT_CREDIT']
    previous_data.loc[previous_data['NAME_CONTRACT_STATUS_Approved'] == 0, 'percentage_down'] = np.nan
    previous_data['AMT_CREDIT_accepted'] = previous_data['AMT_CREDIT']
    previous_data.loc[previous_data['NAME_CONTRACT_STATUS_Approved'] == 0, 'AMT_CREDIT_accepted'] = np.nan
    previous_data['AMT_ANNUITY_accepted'] = previous_data['AMT_ANNUITY']
    previous_data.loc[previous_data['NAME_CONTRACT_STATUS_Approved'] == 0, 'AMT_ANNUITY_accepted'] = np.nan
    
    previous_data['PREVIOUS_APP_LATE_DAYS'] = previous_data['DAYS_FIRST_DUE'] - previous_data['DAYS_FIRST_DRAWING']
    previous_data['PREVIOUS_APP_LAST_DUE_CHANGE'] = previous_data['DAYS_LAST_DUE_1ST_VERSION'] - previous_data['DAYS_LAST_DUE']
    
    for col in ('AMT_ANNUITY', 'AMT_ANNUITY_accepted', 'AMT_APPLICATION', 'AMT_GOODS_PRICE'):
        previous_data[col + '/AMT_CREDIT'] = previous_data[col] / previous_data['AMT_CREDIT']
        
    previous_data = previous_data.replace([float('inf'), -float('inf')], np.nan)
    agg_dict = {'SK_ID_PREV': 'count', 
         'PREVIOUS_APP_LATE_DAYS': ('max', 'mean'),
         'PREVIOUS_APP_LAST_DUE_CHANGE': ('max', 'mean'),
         'LAST_APP_REJECTED': 'max',
         'DAYS_DECISION': ('min', 'max'),
         'NAME_CONTRACT_STATUS_Refused': ('count', 'mean'), 
         'NAME_CONTRACT_STATUS_Approved': ('count', 'mean'),
         'AMT_CREDIT': 'mean',
         'AMT_CREDIT_accepted': ('mean', 'max'),
         'percentage_down': 'mean',
         'CNT_PAYMENT': ('mean', 'max'),
         'NFLAG_INSURED_ON_APPROVAL': 'mean',
         'AMT_ANNUITY': ('mean', 'max'),
         'AMT_ANNUITY_accepted': ('mean', 'max'),
         'AMT_ANNUITY/AMT_CREDIT': ('mean', 'max'),
         'AMT_ANNUITY_accepted/AMT_CREDIT': ('mean', 'max'),
         'AMT_APPLICATION/AMT_CREDIT': ('mean', 'max'),
         'AMT_GOODS_PRICE/AMT_CREDIT': ('mean', 'max'),
	 'AMT_GOODS_PRICE': ('mean')}

    for col in previous_data.columns:
        if col not in agg_dict and col not in ('SK_ID_CURR', 'SK_ID_PREV'):
            agg_dict[col] = 'mean'

    

    previous_data = previous_data.groupby('SK_ID_CURR').agg(agg_dict)
    previous_data.columns = list(map('_'.join, previous_data.columns.values))

    previous_data = previous_data.reset_index()

    return previous_data

%time previous_data = process_previous_application_data(previous_data)
previous_data.fillna(0)
previous_data.head()

CPU times: user 8.04 s, sys: 532 ms, total: 8.57 s
Wall time: 8.59 s
CPU times: user 58.9 s, sys: 9.88 s, total: 1min 8s
Wall time: 1min 8s


,SK_ID_CURR,NAME_GOODS_CATEGORY_Animals_mean,WEEKDAY_APPR_PROCESS_START_FRIDAY_mean,NAME_TYPE_SUITE_Unaccompanied_mean,PRODUCT_COMBINATION_Cash X-Sell: high_mean,NAME_GOODS_CATEGORY_Gardening_mean,DAYS_FIRST_DRAWING_mean,HOUR_APPR_PROCESS_START_mean,NAME_CASH_LOAN_PURPOSE_Buying a home_mean,AMT_ANNUITY_accepted/AMT_CREDIT_mean,...,CODE_REJECT_REASON_HC_mean,NAME_GOODS_CATEGORY_Mobile_mean,NAME_CASH_LOAN_PURPOSE_Car repairs_mean,DAYS_LAST_DUE_mean,NAME_CONTRACT_TYPE_Consumer loans_mean,PRODUCT_COMBINATION_POS household with interest_mean,NAME_SELLER_INDUSTRY_Industry_mean,AMT_GOODS_PRICE_mean,CODE_REJECT_REASON_LIMIT_mean,RATE_DOWN_PAYMENT_mean
0,100001.0,0.0,1.000000,0.000000,0.0,0.0,NaN,13.000000,0.0,0.166099,...,0.0,1.0,0.0,-1619.000000,1.000000,0.000000,0.0,24835.5,0.0,0.104326
1,100002.0,0.0,0.000000,0.000000,0.0,0.0,NaN,9.000000,0.0,0.051670,...,0.0,0.0,0.0,-25.000000,1.000000,0.000000,0.0,179055.0,0.0,0.000000
2,100003.0,0.0,0.333333,0.333333,0.0,0.0,NaN,14.666667,0.0,0.126383,...,0.0,0.0,0.0,-1054.333333,0.666667,0.333333,0.0,435436.5,0.0,0.050030
3,100004.0,0.0,1.000000,1.000000,0.0,0.0,NaN,5.000000,0.0,0.266450,...,0.0,1.0,0.0,-724.000000,1.000000,0.000000,0.0,24282.0,0.0,0.212008
4,100005.0,0.0,0.500000,0.000000,0.0,0.0,NaN,10.500000,0.0,0.119870,...,0.0,0.5,0.0,-466.000000,0.500000,0.000000,0.0,44617.5,0.0,0.108964


In [4]:
%time data_bureau = pd.read_csv('data/bureau.csv')
bureau_balance = pd.read_csv('data/bureau_balance.csv')

def process_bureau(data_bureau,bureau_balance):
    
    bureau_balance = pd.get_dummies(bureau_balance,columns = ['STATUS'])

    list_replacement_0 = [-i for i in range(12)]
    list_replacement_1 = [-i for i in range(12,97)]

    bureau_balance['MONTHS_BALANCE'] = bureau_balance['MONTHS_BALANCE'].replace(list_replacement_0,0)
    bureau_balance['MONTHS_BALANCE'] = bureau_balance['MONTHS_BALANCE'].replace(list_replacement_1,1)

    bureau_balance = bureau_balance.groupby('SK_ID_BUREAU', as_index = False).agg(
        {'MONTHS_BALANCE':'mean',
         'STATUS_0':'mean',
         'STATUS_1':'mean',
         'STATUS_2':'mean',
         'STATUS_3':'mean',
         'STATUS_4':'mean',
         'STATUS_5':'mean',
         'STATUS_C':'mean',
         'STATUS_X':'mean'})
   
    data_bureau = data_bureau.merge(bureau_balance, on = 'SK_ID_BUREAU',how = 'left')
    
    data_bureau['TOTAL_CREDITS'] = 0
    data_bureau['MAX(DAYS_CREDIT_ENDDATE, 0)'] = data_bureau['DAYS_CREDIT_ENDDATE']
    data_bureau.loc[data_bureau['MAX(DAYS_CREDIT_ENDDATE, 0)'] < 0, 'MAX(DAYS_CREDIT_ENDDATE, 0)'] = 0
    
    data_bureau = data_bureau.replace(['Mobile operator loan','Interbank credit','Loan for purchase of shares (margin lending)','Loan for purchase of equipment','Another type of loan','Unknown type of loan'],'Other')

    data_bureau = pd.get_dummies(data_bureau,columns = ['CREDIT_ACTIVE','CREDIT_CURRENCY','CREDIT_TYPE'])
    data_bureau = data_bureau.drop(['DAYS_CREDIT_UPDATE','CREDIT_CURRENCY_currency 1','CREDIT_CURRENCY_currency 2','CREDIT_CURRENCY_currency 3','CREDIT_CURRENCY_currency 4'],axis=1)
    data_bureau = data_bureau.fillna(data_bureau.median())
    for column in data_bureau.columns:
        if column.startswith('AMT_'):
            data_bureau[column + '/' + 'AMT_CREDIT_SUM'] = data_bureau[column] / data_bureau['AMT_CREDIT_SUM']
            
    data_bureau = data_bureau.replace([float('inf'), -float('inf')], np.nan)
    data_bureau = data_bureau.groupby('SK_ID_CURR').agg(
        {'SK_ID_BUREAU':'count',
         'DAYS_CREDIT':('min', 'max'),
         'CREDIT_DAY_OVERDUE':'max',
         'DAYS_CREDIT_ENDDATE':('min', 'max'),
         'MAX(DAYS_CREDIT_ENDDATE, 0)': ('sum', 'max'),
         'DAYS_ENDDATE_FACT':'min',
         'AMT_CREDIT_MAX_OVERDUE':['max','mean'],
         'AMT_CREDIT_MAX_OVERDUE/AMT_CREDIT_SUM':['max','mean'],
         'CNT_CREDIT_PROLONG':['max','median'],
         'AMT_CREDIT_SUM':['max','median'],
         'AMT_CREDIT_SUM_DEBT':'sum',
         'AMT_CREDIT_SUM_DEBT/AMT_CREDIT_SUM':['max', 'mean'],
         'AMT_CREDIT_SUM_LIMIT':'max',
         'AMT_CREDIT_SUM_LIMIT/AMT_CREDIT_SUM':['max', 'mean'],
         'AMT_CREDIT_SUM_OVERDUE':'sum',
         'AMT_CREDIT_SUM_OVERDUE/AMT_CREDIT_SUM':['max', 'mean'],
         'CREDIT_ACTIVE_Active':['count', 'mean'],
         'CREDIT_ACTIVE_Bad debt':['count', 'mean'],
         'CREDIT_ACTIVE_Closed':['count', 'mean'],
         'CREDIT_ACTIVE_Sold':['count', 'mean'],
         'CREDIT_TYPE_Car loan':'mean',
         'CREDIT_TYPE_Cash loan (non-earmarked)':'mean',
         'CREDIT_TYPE_Consumer credit':'mean',
         'CREDIT_TYPE_Credit card':'mean',
         'CREDIT_TYPE_Loan for business development':'mean',
         'CREDIT_TYPE_Loan for the purchase of equipment':'mean',
         'CREDIT_TYPE_Loan for working capital replenishment':'mean',
         'CREDIT_TYPE_Microloan':'mean',
         'CREDIT_TYPE_Mortgage':'mean',
         'CREDIT_TYPE_Other':'mean',
         'CREDIT_TYPE_Real estate loan':'mean',
         'AMT_ANNUITY':['max', 'mean'],
         'AMT_ANNUITY/AMT_CREDIT_SUM':['max', 'mean'],
         'TOTAL_CREDITS': 'count'})

    data_bureau.columns = list(map('_'.join, data_bureau.columns.values))
    data_bureau = data_bureau.reset_index()
    
    
    return data_bureau

%time data_bureau = process_bureau(data_bureau,bureau_balance)
del bureau_balance
gc.collect()
data_bureau.head()

CPU times: user 3.16 s, sys: 200 ms, total: 3.36 s
Wall time: 3.39 s
CPU times: user 39.1 s, sys: 6.57 s, total: 45.7 s
Wall time: 45.8 s


,SK_ID_CURR,CREDIT_TYPE_Microloan_mean,CREDIT_ACTIVE_Closed_count,CREDIT_ACTIVE_Closed_mean,"MAX(DAYS_CREDIT_ENDDATE, 0)_sum","MAX(DAYS_CREDIT_ENDDATE, 0)_max",CREDIT_ACTIVE_Sold_count,CREDIT_ACTIVE_Sold_mean,DAYS_CREDIT_ENDDATE_min,DAYS_CREDIT_ENDDATE_max,...,CREDIT_TYPE_Credit card_mean,AMT_CREDIT_MAX_OVERDUE/AMT_CREDIT_SUM_max,AMT_CREDIT_MAX_OVERDUE/AMT_CREDIT_SUM_mean,DAYS_ENDDATE_FACT_min,AMT_ANNUITY/AMT_CREDIT_SUM_max,AMT_ANNUITY/AMT_CREDIT_SUM_mean,CREDIT_TYPE_Other_mean,CREDIT_TYPE_Loan for business development_mean,AMT_CREDIT_SUM_max,AMT_CREDIT_SUM_median
0,100001,0.0,7,0.571429,3091.0,1778.0,7,0.0,-1329.0,1778.0,...,0.000000,0.000000,0.000000,-1328.0,0.055627,0.013996,0.0,0.0,378000.0,168345.00
1,100002,0.0,8,0.750000,927.0,780.0,8,0.0,-1072.0,780.0,...,0.500000,0.174139,0.031026,-1185.0,0.000000,0.000000,0.0,0.0,450000.0,54130.50
2,100003,0.0,4,0.750000,1216.0,1216.0,4,0.0,-2434.0,1216.0,...,0.500000,0.000000,0.000000,-2131.0,0.000000,0.000000,0.0,0.0,810000.0,92576.25
3,100004,0.0,2,1.000000,0.0,0.0,2,0.0,-595.0,-382.0,...,0.000000,0.000000,0.000000,-683.0,0.000000,0.000000,0.0,0.0,94537.8,94518.90
4,100005,0.0,3,0.333333,1446.0,1324.0,3,0.0,-128.0,1324.0,...,0.333333,0.000000,0.000000,-897.0,0.142879,0.047626,0.0,0.0,568800.0,58500.00


In [5]:
%time data_pos_cash = pd.read_csv('data/POS_CASH_balance.csv')
def process_poscash(data_pos_cash):
    data_pos_cash['NAME_CONTRACT_STATUS'] = data_pos_cash['NAME_CONTRACT_STATUS'].replace(['Signed','Demand','Returned to the store','Approved','XNA'],0)
    data_pos_cash['NAME_CONTRACT_STATUS'] = data_pos_cash['NAME_CONTRACT_STATUS'].replace({'Amortized debt': 1,'Active':3,'Canceled':2,'Completed':4})

    data_pos_cash.loc[(data_pos_cash['NAME_CONTRACT_STATUS'].isin((1, 3))) & (data_pos_cash['CNT_INSTALMENT_FUTURE'] == 0), 'NAME_CONTRACT_STATUS'] = 4
    data_pos_cash.loc[(data_pos_cash['NAME_CONTRACT_STATUS'].isin((1, 3))) & (data_pos_cash['CNT_INSTALMENT_FUTURE'] != 0), 'NAME_CONTRACT_STATUS'] = 2

    data_pos_cash['MONTHS_BALANCE'] = data_pos_cash['MONTHS_BALANCE'].replace(-1, 0)

    data_pos_cash['MONTHS_LEFT_TO_PAY'] = data_pos_cash['CNT_INSTALMENT_FUTURE']
    data_pos_cash.loc[data_pos_cash['MONTHS_BALANCE'] != 0, 'MONTHS_LEFT_TO_PAY'] = np.nan 
    
    data_pos_cash = data_pos_cash.sort_values(['SK_ID_CURR', 'SK_ID_PREV', 'MONTHS_BALANCE'])
    data_pos_cash['CNT_INSTALMENT_DIFF'] = data_pos_cash['CNT_INSTALMENT'].diff()

    data_pos_cash.loc[data_pos_cash['SK_ID_PREV'].diff() != 0, 'CNT_INSTALMENT_DIFF'] = 0

    aggregated = data_pos_cash.groupby(('SK_ID_CURR', 'SK_ID_PREV')).agg({
        'SK_DPD_DEF': ('sum', 'max'),
        'SK_DPD': ('sum', 'max'),
        'MONTHS_LEFT_TO_PAY': ('max'),
        'CNT_INSTALMENT_FUTURE': ('mean', 'min'),
        'CNT_INSTALMENT': ('mean', 'max'),
        'NAME_CONTRACT_STATUS': 'max',
        'CNT_INSTALMENT_DIFF': 'sum',
        'CNT_INSTALMENT': 'mean'
    }).fillna(0)

    agg_dict = {col: ('mean', 'max') for col in aggregated.columns}
    agg_dict[('NAME_CONTRACT_STATUS', 'max')] = ('min')
    agg_dict[('CNT_INSTALMENT_DIFF', 'sum')] = ('mean', 'min', 'max')

    aggregated = aggregated.groupby('SK_ID_CURR').agg(agg_dict)


    aggregated.columns = list(map('_'.join, aggregated.columns.values))
    aggregated = aggregated.reset_index()
    return aggregated

%time data_pos_cash = process_poscash(data_pos_cash)
data_pos_cash.head()

CPU times: user 6.86 s, sys: 236 ms, total: 7.1 s
Wall time: 7.12 s
CPU times: user 17 s, sys: 872 ms, total: 17.9 s
Wall time: 17.9 s


,SK_ID_CURR,SK_DPD_max_mean,SK_DPD_max_max,CNT_INSTALMENT_FUTURE_min_mean,CNT_INSTALMENT_FUTURE_min_max,MONTHS_LEFT_TO_PAY_max_mean,MONTHS_LEFT_TO_PAY_max_max,CNT_INSTALMENT_FUTURE_mean_mean,CNT_INSTALMENT_FUTURE_mean_max,SK_DPD_DEF_max_mean,...,SK_DPD_DEF_sum_mean,SK_DPD_DEF_sum_max,CNT_INSTALMENT_mean_mean,CNT_INSTALMENT_mean_max,SK_DPD_sum_mean,SK_DPD_sum_max,NAME_CONTRACT_STATUS_max_min,CNT_INSTALMENT_DIFF_sum_mean,CNT_INSTALMENT_DIFF_sum_min,CNT_INSTALMENT_DIFF_sum_max
0,100001,3.5,7,0.000000,0.0,0.0,0.0,1.375000,2.000,3.5,...,3.5,7,4.000000,4.00,3.5,7,4,0.000000,0.0,0.0
1,100002,0.0,0,6.000000,6.0,6.0,6.0,15.000000,15.000,0.0,...,0.0,0,24.000000,24.00,0.0,0,2,0.000000,0.0,0.0
2,100003,0.0,0,0.333333,1.0,0.0,0.0,5.666667,7.875,0.0,...,0.0,0,9.791667,12.00,0.0,0,2,-1.666667,-5.0,0.0
3,100004,0.0,0,0.000000,0.0,0.0,0.0,2.250000,2.250,0.0,...,0.0,0,3.750000,3.75,0.0,0,4,-1.000000,-1.0,-1.0
4,100005,0.0,0,0.000000,0.0,0.0,0.0,7.200000,7.200,0.0,...,0.0,0,11.700000,11.70,0.0,0,4,-3.000000,-3.0,-3.0


In [6]:
%time data_cc = pd.read_csv('data/credit_card_balance.csv')

def process_cc(data_cc):
    data_cc['ACTIVE_DURING_LAST_YEAR'] = 0
    data_cc.loc[data_cc['MONTHS_BALANCE'] > -12, 'ACTIVE_DURING_LAST_YEAR'] = 1
    data_cc.loc[data_cc['AMT_BALANCE'] == 0, 'ACTIVE_DURING_LAST_YEAR'] = 0
    data_cc['AMT_BALANCE_CURRENT'] = data_cc['AMT_BALANCE']
    data_cc.loc[data_cc['MONTHS_BALANCE'] >= -1, 'AMT_BALANCE_CURRENT'] = np.nan
    
    data_cc['AMT_OVERPAID'] =  (data_cc['AMT_PAYMENT_CURRENT'] / data_cc['AMT_INST_MIN_REGULARITY']).replace(float('inf'), np.nan).fillna(1.0)
    
    aggregated = data_cc.groupby(('SK_ID_CURR', 'SK_ID_PREV')).agg({
        'ACTIVE_DURING_LAST_YEAR': 'max',
        'AMT_BALANCE': ('max', 'mean', 'min'),
        'AMT_BALANCE_CURRENT': ('max'),
        'AMT_CREDIT_LIMIT_ACTUAL': ('max'),
        'AMT_DRAWINGS_ATM_CURRENT': ('max', 'mean'),
        'AMT_DRAWINGS_CURRENT': ('max', 'mean'),
        'AMT_DRAWINGS_OTHER_CURRENT': ('max', 'mean'),
        'AMT_DRAWINGS_POS_CURRENT': ('max', 'mean'),
        'AMT_INST_MIN_REGULARITY': ('max', 'mean'),
        'AMT_PAYMENT_CURRENT': ('max', 'mean'),
        'AMT_PAYMENT_TOTAL_CURRENT': ('max', 'mean'),
        'AMT_RECEIVABLE_PRINCIPAL': ('max', 'mean'),
        'AMT_RECIVABLE': ('max', 'mean'),
        'AMT_TOTAL_RECEIVABLE': ('max', 'mean'),
        'CNT_DRAWINGS_ATM_CURRENT': ('max', 'mean'),
        'CNT_DRAWINGS_CURRENT': ('max', 'mean'),
        'CNT_DRAWINGS_OTHER_CURRENT': ('max', 'mean'),
        'CNT_DRAWINGS_POS_CURRENT': ('max', 'mean'),
        'CNT_INSTALMENT_MATURE_CUM': ('max', 'mean'),
        'SK_DPD': ('max', 'sum', 'mean'),
        'SK_DPD_DEF': ('max', 'sum', 'mean'),
        'MONTHS_BALANCE': ('min'),
        'AMT_OVERPAID': ('mean', 'min', 'max')
    })
    agg_dict = {col: ('mean') for col in aggregated.columns}
    agg_dict[('ACTIVE_DURING_LAST_YEAR', 'max')] = ('sum', 'mean', 'count')
    agg_dict[('AMT_BALANCE_CURRENT', 'max')] = ('sum', 'mean', 'max')
    agg_dict[('MONTHS_BALANCE', 'min')] = ('mean', 'min')
    aggregated = aggregated.groupby('SK_ID_CURR').agg(agg_dict)
    aggregated.columns = list(map('_'.join, aggregated.columns.values))
    aggregated = aggregated.reset_index()
        
    return aggregated

%time data_cc = process_cc(data_cc)
data_cc.head()

CPU times: user 8.84 s, sys: 404 ms, total: 9.24 s
Wall time: 9.25 s
CPU times: user 4.14 s, sys: 780 ms, total: 4.92 s
Wall time: 4.92 s


,SK_ID_CURR,CNT_DRAWINGS_OTHER_CURRENT_mean_mean,AMT_PAYMENT_CURRENT_max_mean,AMT_RECEIVABLE_PRINCIPAL_mean_mean,SK_DPD_DEF_max_mean,ACTIVE_DURING_LAST_YEAR_max_sum,ACTIVE_DURING_LAST_YEAR_max_mean,ACTIVE_DURING_LAST_YEAR_max_count,AMT_RECEIVABLE_PRINCIPAL_max_mean,AMT_OVERPAID_min_mean,...,AMT_DRAWINGS_CURRENT_max_mean,AMT_DRAWINGS_CURRENT_mean_mean,AMT_BALANCE_mean_mean,CNT_DRAWINGS_ATM_CURRENT_max_mean,AMT_RECIVABLE_mean_mean,AMT_BALANCE_min_mean,CNT_DRAWINGS_CURRENT_mean_mean,CNT_DRAWINGS_POS_CURRENT_mean_mean,AMT_PAYMENT_TOTAL_CURRENT_mean_mean,AMT_INST_MIN_REGULARITY_mean_mean
0,100006,NaN,NaN,0.000000,0.0,0,0.0,1,0.0,1.0,...,0.0,0.000000,0.000000,NaN,0.000000,0.0,0.000000,NaN,0.000000,0.000000
1,100011,0.0,55485.0,52402.088919,0.0,0,0.0,1,180000.0,1.0,...,180000.0,2432.432432,54482.111149,4.0,54433.179122,0.0,0.054054,0.0,4520.067568,3956.221849
2,100013,0.0,153675.0,17255.559844,1.0,0,0.0,1,157500.0,0.0,...,157500.0,5953.125000,18159.919219,7.0,18101.079844,0.0,0.239583,0.0,6817.172344,1454.539551
3,100021,NaN,NaN,0.000000,0.0,0,0.0,1,0.0,1.0,...,0.0,0.000000,0.000000,NaN,0.000000,0.0,0.000000,NaN,0.000000,0.000000
4,100023,NaN,NaN,0.000000,0.0,0,0.0,1,0.0,1.0,...,0.0,0.000000,0.000000,NaN,0.000000,0.0,0.000000,NaN,0.000000,0.000000


In [7]:
%time data_installments = pd.read_csv('data/installments_payments.csv')

def process_installments(data_installments):
    data_installments = data_installments[pd.notna(data_installments['DAYS_ENTRY_PAYMENT'])]

    data_installments['DIFERENCE_DAYS_PAYMENT'] = data_installments['DAYS_ENTRY_PAYMENT'] - data_installments['DAYS_INSTALMENT']
    data_installments['DIFERENCE_PAYMENT'] = data_installments['AMT_INSTALMENT'] - data_installments['AMT_PAYMENT']

    data_installments['DAYS_INSTALMENT_new'] = 0
    data_installments['DAYS_ENTRY_PAYMENT_new'] = 0

    data_installments.loc[data_installments['DAYS_INSTALMENT'] < -365 , 'DAYS_INSTALMENT_new'] = 1
    data_installments.loc[data_installments['DAYS_ENTRY_PAYMENT'] < -365, 'DAYS_ENTRY_PAYMENT_new'] = 1

    data_installments = data_installments.drop(['DAYS_INSTALMENT','DAYS_ENTRY_PAYMENT'],axis = 1)
    data_installments = data_installments.rename(columns = {'DAYS_INSTALMENT_new':'DAYS_INSTALMENT','DAYS_ENTRY_PAYMENT_new':'DAYS_ENTRY_PAYMENT'})

    data_installments = data_installments.groupby(by=['SK_ID_PREV','SK_ID_CURR']).agg(
        {'DAYS_INSTALMENT':'mean',
         'DAYS_ENTRY_PAYMENT':'mean',
         'AMT_INSTALMENT':'median',
         'AMT_PAYMENT':'median',
         'NUM_INSTALMENT_VERSION': 'max', 
         'DIFERENCE_DAYS_PAYMENT': 'mean',
         'DIFERENCE_PAYMENT': 'mean'})
    data_installments = data_installments.groupby(by='SK_ID_CURR').agg(
        {'DAYS_INSTALMENT':'mean',
         'DAYS_ENTRY_PAYMENT':'mean',
         'AMT_INSTALMENT':'median',
         'AMT_PAYMENT':'median',
         'NUM_INSTALMENT_VERSION': 'mean', 
         'DIFERENCE_DAYS_PAYMENT': 'mean',
         'DIFERENCE_PAYMENT': 'mean'})
    #     data_installments.columns = list(map('_'.join, data_installments.columns.values))
    data_installments = data_installments.reset_index()
    return data_installments

%time data_installments = process_installments(data_installments)
data_installments.head()

CPU times: user 11.8 s, sys: 816 ms, total: 12.6 s
Wall time: 12.6 s


/home/goncalo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/goncalo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/goncalo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

CPU times: user 12.1 s, sys: 2.3 s, total: 14.4 s
Wall time: 14.4 s


,SK_ID_CURR,DAYS_ENTRY_PAYMENT,AMT_PAYMENT,DIFERENCE_DAYS_PAYMENT,DIFERENCE_PAYMENT,AMT_INSTALMENT,NUM_INSTALMENT_VERSION,DAYS_INSTALMENT
0,100001,1.000000,3966.525,-5.916667,0.0,3966.525,1.500000,1.000000
1,100002,0.421053,9251.775,-20.421053,0.0,9251.775,2.000000,0.368421
2,100003,1.000000,64567.665,-7.448413,0.0,64567.665,1.333333,1.000000
3,100004,1.000000,5357.250,-7.666667,0.0,5357.250,2.000000,1.000000
4,100005,1.000000,4813.200,-23.555556,0.0,4813.200,2.000000,1.000000


In [8]:
%time
data = data.merge(previous_data, on='SK_ID_CURR', how='left', suffixes=('', '_previous'))
data_submission = data_submission.merge(previous_data, on='SK_ID_CURR', how='left', suffixes=('', '_previous'))
data = data.merge(data_bureau, on= 'SK_ID_CURR',how='left', suffixes=('', '_bureau'))
data_submission = data_submission.merge(data_bureau, on= 'SK_ID_CURR',how='left', suffixes=('', '_bureau'))
data = data.merge(data_pos_cash, on= 'SK_ID_CURR',how='left', suffixes=('', '_bureau'))
data_submission = data_submission.merge(data_pos_cash, on= 'SK_ID_CURR',how='left', suffixes=('', '_poscash'))
data = data.merge(data_cc, on= 'SK_ID_CURR',how='left', suffixes=('', '_cc'))
data_submission = data_submission.merge(data_cc, on= 'SK_ID_CURR',how='left', suffixes=('', '_cc'))
data = data.merge(data_installments, on= 'SK_ID_CURR',how='left', suffixes=('', '_installments'))
data_submission = data_submission.merge(data_installments, on= 'SK_ID_CURR',how='left', suffixes=('', '_installments'))


data = data.drop('SK_ID_CURR', axis=1)
data_submission = data_submission.drop('SK_ID_CURR', axis=1)

# manual features
data['AMT_PREVIOUS_CREDIT/AMT_CREDIT'] = data['AMT_CREDIT_accepted_mean'] / data['AMT_CREDIT']
data_submission['AMT_PREVIOUS_CREDIT/AMT_CREDIT'] = data_submission['AMT_CREDIT_accepted_mean'] / data_submission['AMT_CREDIT']

data['AMT_PREVIOUS_MAXCREDIT/AMT_CREDIT'] = data['AMT_CREDIT_accepted_max'] / data['AMT_CREDIT']
data_submission['AMT_PREVIOUS_MAXCREDIT/AMT_CREDIT'] = data_submission['AMT_CREDIT_accepted_max'] / data_submission['AMT_CREDIT']

data['AMT_PREVIOUS_ANNUITY/AMT_ANNUITY'] = data['AMT_ANNUITY_accepted_mean'] / data['AMT_ANNUITY']
data_submission['AMT_PREVIOUS_ANNUITY/AMT_ANNUITY'] = data_submission['AMT_ANNUITY_accepted_mean'] / data_submission['AMT_ANNUITY']

data['AMT_BUREAU_ANNUITY/AMT_ANNUITY'] = data['AMT_ANNUITY_mean'] / data['AMT_ANNUITY']
data_submission['AMT_BUREAU_ANNUITY/AMT_ANNUITY'] = data_submission['AMT_ANNUITY_mean'] / data['AMT_ANNUITY']

data['AMT_BUREAU_CREDIT/AMT_CREDIT'] = data['AMT_CREDIT_SUM_max'] / data['AMT_CREDIT']
data_submission['AMT_BUREAU_CREDIT/AMT_CREDIT'] = data_submission['AMT_CREDIT_SUM_max'] / data['AMT_CREDIT']

# data['AMT_PREVIOUS_GOODS_PRICE/AMT_GOODS_PRICE'] = data['AMT_GOODS_PRICE/AMT_CREDIT_mean'] / data['AMT_GOODS_PRICE/(AMT_CREDIT)']
# data_submission['AMT_PREVIOUS_GOODS_PRICE/AMT_GOODS_PRICE'] = data_submission['AMT_GOODS_PRICE/AMT_CREDIT_mean'] / data_submission['AMT_GOODS_PRICE/(AMT_CREDIT)']


# ## BUREAU FILLING NA
# default_values = dict()

# for column in ['DAYS_CREDIT_min','DAYS_ENDDATE_FACT_min','AMT_CREDIT_SUM_LIMIT_max','DAYS_CREDIT_ENDDATE_max']:
#     default_values[column] = data_bureau[column].median()
    
# data = data.fillna(default_values)
# data = data.fillna(0)
# data_submission= data_submission.fillna(default_values)
# data_submission= data_submission.fillna(0)

## PREVIOUS APPLICATION FILLING NA

default_values = {
    'AMT_CREDIT_mean': 0,
    'AMT_CREDIT_accepted_max': 0,
    'AMT_CREDIT_accepted_mean': 0,
    'NAME_CONTRACT_STATUS_Approved_mean': 1.0,
    'NAME_CONTRACT_STATUS_Refused_mean': 0.0,
    'SK_ID_PREV_count': 0
}

# data = data.fillna(default_values)
# data = data.fillna(data.median())

# data_submission = data_submission.fillna(default_values)
# data_submission = data_submission.fillna(data.median())

del previous_data, data_bureau, data_pos_cash, data_cc, data_installments
gc.collect()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.15 µs


588

In [9]:
print(set(data.columns) - set(data_submission.columns))
data_submission['NAME_INCOME_TYPE_Maternity leave'] = 0

set(['NAME_INCOME_TYPE_Maternity leave', 'TARGET'])


In [ ]:
all_columns = list(set(data.columns) - {'TARGET'})
data_x = data.as_matrix(all_columns)
data_y = data['TARGET'].as_matrix()
data_submission_x = data_submission.as_matrix(all_columns)

train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.05)
del data_x, data_y

In [ ]:
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)
data_submission_x = scaler.transform(data_submission_x)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_x, train_y)

In [ ]:
roc_auc_score(test_y, lr.predict_proba(test_x)[:,1])

In [ ]:
xgb = XGBClassifier()
xgb.fit(train_x, train_y)

In [ ]:
print('train_score:', roc_auc_score(train_y, xgb.predict_proba(train_x)[:,1]))
print('test_score:', roc_auc_score(test_y, xgb.predict_proba(test_x)[:,1]))

In [ ]:
print('TOP 20 Importances')
for importance, col in sorted(zip(list(xgb.feature_importances_), list(all_columns)), reverse=True)[:20]:
    print(str(importance) + '\t' + col)

In [ ]:
predicted = xgb.predict_proba(data_submission_x)[:, 1]

results = pd.DataFrame({'SK_ID_CURR': submission_id, 'TARGET': predicted})

results.to_csv('predictions.csv', index=False)

In [10]:
# #del train_x, train_y, test_x, test_y, data_submission_x
data_y = data['TARGET']
data.drop('TARGET', axis=1, inplace=True, errors='ignore')
from script import train_model
folds = KFold(n_splits=5, shuffle=True, random_state=546789)
# Train model and get oof and test predictions
data_submission['SK_ID_CURR'] = submission_id
oof_preds, test_preds, importances = train_model(data, data_submission, data_y, folds)

/usr/local/lib/python2.7/dist-packages/statsmodels/nonparametric/kde.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .linbin import fast_linbin
/usr/local/lib/python2.7/dist-packages/statsmodels/nonparametric/kde.py:26: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from .linbin import fast_linbin
/usr/local/lib/python2.7/dist-packages/statsmodels/nonparametric/smoothers_lowess.py:11: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._smoothers_lowess import lowess as _lowess
/usr/local/lib/python2.7/dist-packages/statsmodels/nonparametric/smoothers_lowess.py:11: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from ._smoothers_lowess import lowess as _lowess
/home/goncalo/.local/lib/python2.7/site-packages/scipy/cluster/vq.py:78: RuntimeWarning: numpy.dtype

Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.789277	valid_1's auc: 0.767841
[200]	training's auc: 0.810911	valid_1's auc: 0.780097
[300]	training's auc: 0.825578	valid_1's auc: 0.785012
[400]	training's auc: 0.837436	valid_1's auc: 0.787451
[500]	training's auc: 0.847576	valid_1's auc: 0.788174
[600]	training's auc: 0.857199	valid_1's auc: 0.788772
[700]	training's auc: 0.865587	valid_1's auc: 0.788891
[800]	training's auc: 0.873245	valid_1's auc: 0.78891
[900]	training's auc: 0.880516	valid_1's auc: 0.789004
[1000]	training's auc: 0.887222	valid_1's auc: 0.789159
[1100]	training's auc: 0.893936	valid_1's auc: 0.789122
Early stopping, best iteration is:
[1014]	training's auc: 0.888109	valid_1's auc: 0.789214
Fold  1 AUC : 0.789214
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.789061	valid_1's auc: 0.77194
[200]	training's auc: 0.810962	valid_1's auc: 0.782615
[300]	training's auc: 0.825485	valid_1's auc:

In [11]:
test_preds.to_csv('predictions.csv', index=False)

In [12]:
importances.sort_values('importance')

,feature,importance,fold
227,ORGANIZATION_TYPE_Transport: type 2,0,3
197,ORGANIZATION_TYPE_Industry: type 6,0,5
234,FONDKAPREMONT_MODE_reg oper spec account,0,2
235,HOUSETYPE_MODE_block of flats,0,2
199,ORGANIZATION_TYPE_Industry: type 8,0,5
237,HOUSETYPE_MODE_terraced house,0,2
238,WALLSMATERIAL_MODE_Block,0,2
201,ORGANIZATION_TYPE_Insurance,0,5
338,NAME_GOODS_CATEGORY_Weapon_mean,0,1
239,WALLSMATERIAL_MODE_Mixed,0,2


In [13]:
from script import display_importances, display_precision_recall, display_roc_curve
display_importances(feature_importance_df_=importances)

/home/goncalo/.local/lib/python2.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
